# k-NN
### (Implementação usada na lista 01)

In [1]:
import pandas as pd
import numpy as np

In [2]:
def euclidean(ai, bi):
    return ((float(ai) - float(bi)))**2

In [3]:
def calcDistance(query, instance, weighted):
    distance = 0
    weight = 1
    for i in range(len(instance)-1):
        distance = distance + euclidean(query[i],instance[i])
    distance = (distance)**0.5
    if (weighted == 1):
        if (distance==0):
            distance = 0.01
        weight = 1/(distance**2)
    return ((distance, weight))

In [4]:
def vdm(ai, bi, attributeCount, pos, classList):
    nia = 0
    nib = 0
    vdmResult = 0
    for i in range(len(attributeCount)):
        if (attributeCount[i][0] == ai and attributeCount[i][2] == pos):
            nia = nia + attributeCount[i][3]
        if (attributeCount[i][0] == bi and attributeCount[i][2] == pos):
            nib = nib + attributeCount[i][3]
                   
    for j in range(len(classList)):
        niac = 0
        nibc = 0
        for i in range(len(attributeCount)):
            if (attributeCount[i][0] == ai and attributeCount[i][1] == classList[j] and attributeCount[i][2] == pos):
                niac = niac + attributeCount[i][3]
            if (attributeCount[i][0] == ai and attributeCount[i][1] == classList[j] and attributeCount[i][2] == pos):
                nibc = nibc + attributeCount[i][3]
        
        if (nia == 0):
            v1 = 0
        else:
            v1 = niac/nia
        
        if (nib == 0):
            v2 = 0
        else:
            v2 = nibc/nib
        
        calc = abs(v1-v2)
        vdmResult = vdmResult + calc
        
    return vdmResult

In [5]:
def calcVDM(query, instance, weighted, attributeCount, classList):
    distance = 0
    weight = 1
    for i in range(len(query)):
        distance = distance + vdm(query[i], instance[i], attributeCount, i, classList)
    
    distance = (distance)**0.5
    
    if (weighted == 1):
        if (distance==0):
            distance = 0.01
        weight = 1/(distance**2)
    
    return ((distance, weight))

In [6]:
def calcHVDM(query, instance, weighted, attributeCount, classList, typeList):
    distance = 0
    weight = 1
    for i in range(len(query)):
        if (typeList[i] == 'num'):
            distance = distance + ((euclidean(query[i], instance[i]))**2)
        elif (typeList[i] == 'cat'):
            distance = distance + ((vdm(query[i], instance[i], attributeCount, i, classList))**2)
    
    distance = (distance)**0.5
    
    if (weighted == 1):
        if(distance==0):
            distance = 0.01
        weight = 1/(distance**2)
    
    return ((distance, weight))

In [7]:
def getNeighbors(distances, n):
    classes = []
    classList = []
    for i in range(n):
        myClass = distances[i][0][-1]
        myWeight = distances[i][2]
        if (myClass not in classList):
            classes.append((myClass, myWeight))
            classList.append(myClass)
        else:
            for j in range(len(classes)):
                if (myClass == classes[j][0]):
                    classes[j] = list(classes[j])
                    classes[j][1] = classes[j][1] + myWeight
                    break
    
    classes.sort(key=lambda x: x[1], reverse=True)
    return classes[0][0]

In [8]:
def knn (query, data, weighted, dataType, typeList):
    # k = testInstance
    # 
    #dataType 0 = numérico, 1 = categorico, 2 = heterogeneo
    distances = []
    classes = []
    
    attributeCount = []
    
    classColumn = [row[-1] for row in data]
    classList = []
    
    for i in range(len(classColumn)):
        if (classColumn[i] not in classList):
            classList.append(classColumn[i])
    
    for i in range(len(data[0])-1):
        for j in range(len(data)):
            flag = 0
            for k in range(len(attributeCount)):
                if (data[j][i]==attributeCount[k][0] and data[j][-1]==attributeCount[k][1] and i == attributeCount[k][2]):
                    attributeCount[k][3] = attributeCount[k][3] + 1
                    flag = 1
                
            if(flag == 0):
                attributeCount.append([data[j][i], data[j][-1], i, 1])
            
            #if (data[j][i] not in attr):
             #   if (len(attr)==0):
              #      attributeCount.append([data[j][i], data[j][-1], 1])
               # else:
                #    for k in range(len(attributeCount)):
                 #       if (data[j][i]==attributeCount[k][0] and data[j][-1]==attributeCount[k][1]):
                  #          attributeCount[k][2] = attributeCount[k][2] + 1
                #attr.append([data[j][i], data[j][-1]])

    #euclidean
    if (dataType == 0):
        for i in range(len(data)):
            distance = calcDistance(query, data[i], weighted)
            distances.append((data[i], distance[0], distance[1]))
        distances.sort(key=lambda x: x[0], reverse=True)
        distances.sort(key=lambda x: x[1], reverse=False)
    #vdm
    elif (dataType == 1):
        for i in range(len(data)):
            distance = calcVDM(query, data[i], weighted, attributeCount, classList)
            distances.append((data[i], distance[0], distance[1]))
        distances.sort(key=lambda x: x[0], reverse=True)
        distances.sort(key=lambda x: x[1], reverse=False)
    
    #hvdm
    elif (dataType == 2):
        for i in range(len(data)):
            distance = calcHVDM(query, data[i], weighted, attributeCount, classList, typeList)
            distances.append((data[i], distance[0], distance[1]))
        distances.sort(key=lambda x: x[0], reverse=True)
        distances.sort(key=lambda x: x[1], reverse=False)
    
    return distances

# k-Folds Cross-Validation
### (Implementação usada na lista 01)

In [9]:
import random

In [10]:
def kFolds (dataset, k, shuffle, mySeed):
    
    if (shuffle==1):
        random.seed(mySeed)
        finalDataset = dataset.sample(frac=1, random_state=mySeed)
    else:
        finalDataset = dataset.copy()
        
    splittedDataset = np.array_split(finalDataset, k)
    
    return splittedDataset

In [11]:
def crossValidation(dataset, kFold, shuffle, mySeed, kVariation, weighted, dataType, typeList):
    
    foldHits = []
    for i in range(len(kVariation)):
        foldHits.append(0)
        
    datasetFolds = kFolds(dataset, kFold, shuffle, mySeed)
    for i in range(kFold):
        testset = datasetFolds[i].reset_index(drop=True)
        frames = []
        for j in range(kFold):
            if (not j==i):
                frames.append(datasetFolds[j])
        trainset = pd.concat(frames).reset_index(drop=True)
        hits = 0
        trainsetList = trainset.values.tolist()
        for m in range(len(testset)):
            neighbors = knn(testset.iloc[m], trainsetList, weighted, dataType, typeList)
            for z in range(len(kVariation)):
                prediction = getNeighbors(neighbors, kVariation[z])    
                if (prediction == testset.iloc[m][-1]):
                    foldHits[z] = foldHits[z] + 1
                
    
    for n in range(len(foldHits)):
        foldHits[n] = foldHits[n]/len(dataset)    
    return foldHits

# LVQ1

In [27]:
#criar X protótipos aleatórios(pelo menos 1 de cada classe)
#varrer o dataset aleatoriamente N vezes
    # achar o protótipo m mais próximo da instância x
    # ajustar posição do protótipo m pela regra m' = m + w(t)*[pos(x)-pos(m)]
        # w(t) 0.5 -> 0.01
        
#rodar knn no dataset dos protótipos

# Datasets

In [14]:
dataretrieve = pd.read_csv("..//datasets/dataretrieve.csv", na_values='?')
desharnais = pd.read_csv("..//datasets/desharnais.csv", na_values='?')
dataretrieve = dataretrieve.fillna(0)
desharnais = desharnais.fillna(0)

In [16]:
with open("..//datasets/dataretrieve.txt") as infile:
    dataretrievelist = infile.read()
    dataretrievelist = dataretrievelist.split(',')
with open("..//datasets/desharnais.txt") as infile:
    desharnaislist = infile.read()
    desharnaislist = desharnaislist.split(',')

# Cálculo do Score
### (Implementação usada na lista 01, sem medir o tempo)

In [17]:
kVariation = [1,3]

In [21]:
def calculateScores(dataset, weighted, dataType, typeList):
    scores = []
    #startTime = time.time()
    scores = crossValidation(dataset, 5, 1, 1000, kVariation, weighted, dataType, typeList)
    #finishTime = time.time()
    #totalTime = finishTime - startTime
    return scores #[,totalTime]

# Resultados sem protótipos

In [23]:
dataretrieveNoProt = calculateScores(dataretrieve, 0, 0, dataretrievelist)
dataretrieveNoProtDF = pd.DataFrame([dataretrieveNoProt], columns = (["k1", "k3"]))
dataretrieveNoProtDF.to_csv("..//results/dataretrieveNoProt.csv", index=False)
dataretrieveNoProtDF

,k1,k3
0,0.838462,0.915385


In [24]:
desharnaisNoProt = calculateScores(desharnais, 0, 0, desharnaislist)
desharnaisNoProtDF = pd.DataFrame([desharnaisNoProt], columns = (["k1", "k3"]))
desharnaisNoProtDF.to_csv("..//results/desharnaisNoProt.csv", index=False)
desharnaisNoProtDF

,k1,k3
0,0.530864,0.555556


# Resultados com LVQ1

## Protótipos = nº de classes

## Protótipos = nº de classes + 1

## Protótipos = 2x nº de classes

## Protótipos = 3x nº de classes

# Resultados com LVQ2.1

# Resultados com LVQ3